<a href="https://colab.research.google.com/github/ishank296/Data-Analysis-Using-Pyspark/blob/main/sparkcode_snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://mirrors.ocf.berkeley.edu/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [45]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,DoubleType
from pyspark.sql.functions import col,lit,desc,spark_partition_id,row_number

In [ ]:
!ls

airports.dat.txt    sample_data		       spark-3.1.2-bin-hadoop3.2.tgz
airports_schma.txt  spark-3.1.2-bin-hadoop3.2


In [ ]:
cat airports_schma.txt

Airport_ID	Unique OpenFlights identifier for this airport.
Name	Name of airport. May or may not contain the City name.
City	Main city served by airport. May be spelled differently from Name.
Country	Country or territory where airport is located. See countries.dat to cross-reference to ISO 3166-1 codes.
IATA	3-letter IATA code. Null if not assigned/unknown.
ICAO	4-letter ICAO code.Null if not assigned.
Latitude	Decimal degrees, usually to six significant digits. Negative is South, positive is North.
Longitude	Decimal degrees, usually to six significant digits. Negative is West, positive is East.
Altitude	In feet.
Timezone	Hours offset from UTC. Fractional hours are expressed as decimals, eg. India is 5.5.
DST	Daylight savings time. One of E (Europe), A (US/Canada), S (South America), O (Australia), Z (New Zealand), N (None) or U (Unknown). See also: Help: Time
Tz	database time zone	Timezone in "tz" (Olson) format, eg. "America/Los_Angeles".
Type	Type of the airport. Value "airport" for 

In [ ]:
field_names = list()
with open('airports_schma.txt') as f:
  for i in f.readlines():
     field_names.append(i.split('\t')[0].lower())
field_names

['airport_id',
 'name',
 'city',
 'country',
 'iata',
 'icao',
 'latitude',
 'longitude',
 'altitude',
 'timezone',
 'dst',
 'tz',
 'type',
 'source']

In [ ]:
fields = list()
for i in field_names:
  curr_field= StructField(i,StringType(),True)
  if i == "longitude" or i == "latitude" or i == "altitude":
    curr_field = StructField(i,DoubleType(),True)
  fields.append(curr_field)
fields


[StructField(airport_id,StringType,true),
 StructField(name,StringType,true),
 StructField(city,StringType,true),
 StructField(country,StringType,true),
 StructField(iata,StringType,true),
 StructField(icao,StringType,true),
 StructField(latitude,DoubleType,true),
 StructField(longitude,DoubleType,true),
 StructField(altitude,DoubleType,true),
 StructField(timezone,StringType,true),
 StructField(dst,StringType,true),
 StructField(tz,StringType,true),
 StructField(type,StringType,true),
 StructField(source,StringType,true)]

In [39]:
airports_df = spark.read.format("csv")\
              .schema(StructType(fields))\
              .load("airports.dat.txt")
airports_df.show(5)            

+----------+--------------------+------------+----------------+----+----+------------------+------------------+--------+--------+---+--------------------+-------+-----------+
|airport_id|                name|        city|         country|iata|icao|          latitude|         longitude|altitude|timezone|dst|                  tz|   type|     source|
+----------+--------------------+------------+----------------+----+----+------------------+------------------+--------+--------+---+--------------------+-------+-----------+
|         1|      Goroka Airport|      Goroka|Papua New Guinea| GKA|AYGA|-6.081689834590001|     145.391998291|  5282.0|      10|  U|Pacific/Port_Moresby|airport|OurAirports|
|         2|      Madang Airport|      Madang|Papua New Guinea| MAG|AYMD|    -5.20707988739|     145.789001465|    20.0|      10|  U|Pacific/Port_Moresby|airport|OurAirports|
|         3|Mount Hagen Kagam...| Mount Hagen|Papua New Guinea| HGU|AYMH|-5.826789855957031|144.29600524902344|  5388.0|     

In [38]:
airports_df.count()

7184

**Airports at highest altitude**

In [49]:
from pyspark.sql import Window
windowspec = Window.orderBy(desc('altitude'))
df1=airports_df.select("name","city","country","altitude")\
      .withColumn("rnk",row_number().over(windowspec)).show(5)

+--------------------+---------+-------+--------+---+
|                name|     city|country|altitude|rnk|
+--------------------+---------+-------+--------+---+
|Qamdo Bangda Airport|   Bangda|  China| 14219.0|  1|
| Ngari Gunsa Airport|Shiquanhe|  China| 14022.0|  2|
|El Alto Internati...|   La Paz|Bolivia| 13355.0|  3|
|Capitan Nicolas R...|   Potosi|Bolivia| 12913.0|  4|
|Yushu Batang Airport|    Yushu|  China| 12816.0|  5|
+--------------------+---------+-------+--------+---+
only showing top 5 rows

